In [ ]:
import pandas as pd
import pickle

In [ ]:
# --------------------------
# 4g) Build age_to_popular.pkl
# --------------------------
# Load raw transactions & customers (for true purchase counts by age)
raw_trans = pd.read_csv(
    "/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv",
    usecols=["customer_id","article_id"]
)
raw_cust  = pd.read_csv(
    "/kaggle/input/h-and-m-personalized-fashion-recommendations/customers.csv",
    usecols=["customer_id","age"]
).dropna(subset=["age"])

# zero-pad article_ids to match your preprocessing
def adjust_id(x):
    s = str(x)
    return "0" + s if len(s)==9 else s
raw_trans["article_id"] = raw_trans["article_id"].apply(adjust_id)

# merge to get age on each purchase
df = raw_trans.merge(raw_cust, on="customer_id", how="inner")

# count purchases per (age, article)
counts = (
    df
    .groupby(["age","article_id"])
    .size()
    .reset_index(name="cnt")
)

# for each age, sort by descending cnt → list of article_ids
age_to_pop = {
    int(age): sub.sort_values("cnt", ascending=False)["article_id"].tolist()
    for age, sub in counts.groupby("age")
}

# save
with open("age_to_popular.pkl", "wb") as f:
    pickle.dump(age_to_pop, f)
print("✅ Saved age_to_popular.pkl")


✅ Saved age_to_popular.pkl
